# Creating a Website to Evaluate the Effect of CRT on School Districts

#### By Joshua Moore and Madison Newberry

## The purpose of this project is to try to determine and illustrate a possible correlation between the inclusion, or exclusion, of Critical Race Theory (CRT) curriculum within K-12 schools and quality of education provided. 
### In order to accomplish this, we will have to perform a significant data acquisition task in order to determine whether or not CRT curriculum being taught in schools correlates with the quality of the education that the children are receiving. 
##### (Credit to Christine Ma, who collaborated with Joshua Moore to come up with this idea, but unfortunately dropped the class before being able to do the project.

#### In order to create our desired end-product, we will be utilizing Flask, which is a micro web framework written in Python that will allow us to publish our webapp.

#### Our datasets were taken from a variety of locations. Our .csv file containing all of the US school districts (as well as their location in latitude and longitude) was taken from https://nces.ed.gov/programs/edge/Geographic/SchoolLocations

#### Our data for this project on a state level consisted of only two states: Alabama and Alaska, although the framework for the other 48 states is present. Alabama has banned CRT, Alaska has not.

#### The data from Alabama came from here: https://www.alabamaachieves.org/reports-data/school-data/
#### and the data from Alaska came from here: https://education.alaska.gov/assessments/results

#### So, let's get to it. Our first step involves defining some functions in two separate files, myProjectDataAnalysisFunctions and myProjectFlaskFunctions.

#### The code from myProjectDataAnalysisFunctions is as follows:

In [1]:
# -*- coding: utf-8 -*-
"""
PIC16B 23F Final Project by Joshua Moore and Madison Newberry
"""

import plotly.io as pio
pio.renderers.default="iframe"
import pandas as pd
import sqlite3
from plotly import express as px

conn = sqlite3.connect("schools.db", check_same_thread=False)


def query_schools_database(state):
    """
    query_schools_database uses SQL to read through a database containing relevant school district location.
​    the state name must be the 2 letter code for the state.
    :state: the name of the state to be investigated
    """
    
    cmd = \
    f"""
    SELECT S.state, S.name, S.lat, S.lon
    FROM schools S
    WHERE S.state = "{state}"
    """
    
    return pd.read_sql_query(cmd, conn)

def generate_state_districts_db():
    """
    generates a db to be read through the query_schools_database and create a df for a specific state.
    """
    
    conn = sqlite3.connect("schools.db") # this creates a database called schools.db.
    df_iter = pd.read_csv("All_US_Schools.csv", chunksize = 1000) # reads in our dataframe in digestible chunks of 1000 rows at a time
    if not conn.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='schools';").fetchone(): 
        df_iter = pd.read_csv("All_US_Schools.csv", chunksize=1000)
        for df_chunk in df_iter:
            df_chunk.to_sql("schools", conn, if_exists="append", index=False)
    conn.close() # closes the connection to the database
    
def generate_state_districts_df(state):
    """
    generates a df using the query_schools_database function, connects and closes connection to database
    :state: the name of the state to be investigated
    """
    conn = sqlite3.connect("schools.db") # this connects to the schools.db database
    df = query_schools_database(state) # this creates the df
    conn.close() # closes the connection to the database
    
    return df

def alabama_state_processing():
    """
    reads in a .csv file downloaded from https://www.alabamaachieves.org/reports-data/school-data/
    this file is renamed to "alabama_prof.csv" before this function is called
    results in a .csv file being downloaded to the computer
    """
    prof_df = pd.read_csv("alabama_prof.csv", header=None) # reads in the csv
    prof_df.columns = prof_df.iloc[4] # sets the 4th row as the column headers
    prof_df = prof_df[prof_df["Grade"] == "ALL"] # isolates only results including all student grade levels
    prof_df = prof_df[prof_df["Test Type"] == "Regular Assessment"] # only includes results for regular assessments, not specialized ones
    prof_df["ALL Category Only ***% Proficient"] = pd.to_numeric(prof_df["ALL Category Only ***% Proficient"], errors="coerce") # sets all the values to numbers in the results column
    average_scores = prof_df.groupby("System Name")["ALL Category Only ***% Proficient"].mean().reset_index() # takes the average for each district by grouping together and ungrouping
    average_scores = average_scores.rename(columns={"System Name" : "NAME", "ALL Category Only ***% Proficient": "Average Assessment Proficiency"}) # renames to a universal name for the website
    average_scores.to_csv("alabama_df.csv", index=False) # downloads the csv
    
def alaska_state_processing():
    """
    reads in a .csv file downloaded from https://education.alaska.gov/assessments/results/results2022
    this file is renamed to "AlaskaResults.csv" before this function is called
    results in a .csv file being downloaded to the computer
    """
    assess_df = pd.read_csv("AlaskaResults.csv") # reads in the csv
    assess_df.columns = assess_df.iloc[2] # sets the 2nd row as the column headers
    assess_df = assess_df[assess_df["Population"] == "All"] # isolates only results including all students, not any separated ones
    assess_df = assess_df[assess_df["Grade"] == "All"] # isolates only results including all student grade levels
    assess_df["District Name"] = assess_df["District Name"] + " School District" # adds the words "School District" to the district name
    # the following 4 lines all replace non-numeric values in the results column to allow them to be turned into numbers
    assess_df["At Target/ Advanced Percentage"] = assess_df["At Target/ Advanced Percentage"].astype(str).str.replace("*","0")
    assess_df["At Target/ Advanced Percentage"] = assess_df["At Target/ Advanced Percentage"].astype(str).str.replace("or more","")
    assess_df["At Target/ Advanced Percentage"] = assess_df["At Target/ Advanced Percentage"].astype(str).str.replace("or fewer","")
    assess_df["At Target/ Advanced Percentage"] = assess_df["At Target/ Advanced Percentage"].astype(str).str.replace("%","")
    
    assess_df["At Target/ Advanced Percentage"] = assess_df["At Target/ Advanced Percentage"].astype(float) # turns them into numbers
    assess_df["At Target/ Advanced Percentage"] = pd.to_numeric(assess_df["At Target/ Advanced Percentage"], errors="coerce") # then turns them into numerics for averaging
    average_scores = assess_df.groupby("District Name")["At Target/ Advanced Percentage"].mean().reset_index() # takes the average
    average_scores = average_scores.rename(columns={"District Name" : "NAME", "At Target/ Advanced Percentage": "Average Assessment Proficiency"}) # renames to a universal name for the website
    average_scores.to_csv("alaska_df.csv", index=False) # downloads the csv
    
    
    

#### There's a lot going on here. I'll go through each function.

#### The first function, `query_schools_database`, performs a query of an SQL database called `schools.db`. It produces a dataframe containing information on the schools of an input state. The state must be input as its two-letter codon (i.e. Alabama should be input as AL).

#### The second function, `generate_state_districts_db`, establishes a connection to the `schools.db` database. It does it in such a way that if replicate information is attempted to be input into the database, it is denied, so that the database doesn't fill up with the same information over and over.

#### The third function, `generate_state_districts_df`, establishes a dataframe for a given state by running the SQL query function. It also opens and closes a connection to the schools.db.

#### The fourth function, `alabama_state_processing`, is a function that shows how the alabama school district data was processed. Since each .csv has a different way of organizing their information, each instance of data processing requires a different function.

#### The fifth function, `alaska_state_processing`, is very similar to the alabama function.

#### Now, myProjectFlaskFunctions:

In [2]:
from flask import Flask, render_template, request, session
import pandas as pd
import sqlite3
import plotly.express as px


def state_crt_condition(state):
    """
    retrieve the status of CRT in a given state

    :state: the name of the state to be investigated
    """
    state = state.lower() # removes any casing issues
    state_crt = { # contains all the state information
        "alabama": "No CRT",
        "alaska": "CRT",
        "arizona": "CRT",
        "arkansas": "No CRT",
        "california": "CRT",
        "colorado": "CRT",
        "connecticut": "CRT",
        "delaware": "CRT",
        "florida": "No CRT",
        "georgia": "No CRT",
        "hawaii": "CRT",
        "idaho": "No CRT",
        "illinois": "CRT",
        "indiana": "CRT",
        "iowa": "No CRT",
        "kansas": "CRT",
        "kentucky": "No CRT",
        "louisiana": "CRT",
        "maine": "CRT",
        "maryland": "CRT",
        "massachusetts": "CRT",
        "michigan": "CRT",
        "minnesota": "CRT",
        "mississippi": "No CRT",
        "missouri": "CRT",
        "montana": "No CRT",
        "nebraska": "CRT",
        "nevada": "CRT",
        "new hampshire": "No CRT",
        "new jersey": "CRT",
        "new mexico": "CRT",
        "new york": "CRT",
        "north carolina": "CRT",
        "north dakota": "No CRT",
        "ohio": "CRT",
        "oklahoma": "No CRT",
        "oregon": "CRT",
        "pennsylvania": "CRT",
        "rhode island": "CRT",
        "south carolina": "No CRT",
        "south dakota": "No CRT",
        "tennessee": "No CRT",
        "texas": "No CRT",
        "utah": "No CRT",
        "vermont": "CRT",
        "virginia": "No CRT",
        "washington": "CRT",
        "west virginia": "CRT",
        "wisconsin": "CRT",
        "wyoming": "CRT",
    }
    return state_crt.get(state)
    
def state_code_conversion(state):
    """
    convert the state full name into a 2 letter code

    :state: the name of the state to be investigated
    """
    state = state.lower() # removes any casing issues
    state_codon = { # contains all the state information
        "alabama": "AL",
        "alaska": "AK",
        "arizona": "AZ",
        "arkansas": "AR",
        "california": "CA",
        "colorado": "CO",
        "connecticut": "CT",
        "delaware": "DE",
        "florida": "FL",
        "georgia": "GA",
        "hawaii": "HI",
        "idaho": "ID",
        "illinois": "IL",
        "indiana": "IN",
        "iowa": "IA",
        "kansas": "KS",
        "kentucky": "KY",
        "louisiana": "LA",
        "maine": "ME",
        "maryland": "MD",
        "massachusetts": "MA",
        "michigan": "MI",
        "minnesota": "MN",
        "mississippi": "MS",
        "missouri": "MO",
        "montana": "MT",
        "nebraska": "NE",
        "nevada": "NV",
        "new hampshire": "NH",
        "new jersey": "NJ",
        "new mexico": "NM",
        "new york": "NY",
        "north carolina": "NC",
        "north dakota": "ND",
        "ohio": "OH",
        "oklahoma": "OK",
        "oregon": "OR",
        "pennsylvania": "PA",
        "rhode island": "RI",
        "south carolina": "SC",
        "south dakota": "SD",
        "tennessee": "TN",
        "texas": "TX",
        "utah": "UT",
        "vermont": "VT",
        "virginia": "VA",
        "washington": "WA",
        "west virginia": "WV",
        "wisconsin": "WI",
        "wyoming": "WY",
    }
    return state_codon.get(state)

#### This one looks like it has a lot going on, but it is just two dictionaries essentially.

#### The first function, `state_crt_condition`, takes in a state name and outputs whether or not that state has banned CRT (or, more specifically, if it allows it).

#### The second function, `state_code_conversion`, takes in a state name and outputs its 2 letter codon.

#### Both of these are essential for processing data on our Flask site, which is where we will host the data and create our visualizations.

#### Next, we'll talk about our Flask site, the bulk of our project.

#### The Flask site is hosted through a file called app.py, and it looks like this:

In [5]:
from myProject import *
from flask import Flask, render_template, request, session
import pandas as pd
import sqlite3
import plotly.express as px
import plotly.io as pio
pio.renderers.default="iframe"


app = Flask(__name__) #creates Flask app
app.secret_key = "PIC16B" #sets secret key

myProjectDataAnalysisFunctions.generate_state_districts_db() # generates the database

@app.route('/')
def index(): # renders the initial page
    return render_template('index.html', PageTitle="School Data")

@app.route('/process', methods=['POST'])
def process(): # renders the page based on what state is chosen on the first page
    selected_state = request.form['state'] # retrieves the chosen state
    state_name = selected_state # sets a variable equal to that state
    session["state_name"] = state_name # adds it to the session
    selected_state = myProjectFlaskFunctions.state_code_conversion(selected_state) # converts the state name to its 2 letter codon
    session["selected_state"] = selected_state # adds the codon to the session
    state_crt = myProjectFlaskFunctions.state_crt_condition(state_name) # retrieves the CRT status for the state
    session["state_crt"] = state_crt # adds the CRT status to the session
    schools_data = myProjectDataAnalysisFunctions.generate_state_districts_df(selected_state) # creates a dataframe for the given state from the database
    prof = pd.read_csv(f"{state_name}_df.csv") # reads in the already-formatted state database for the given state
    final_df = pd.merge(schools_data, prof, on='NAME', how='left') # merges the dfs

    # creates a plotly figure
    fig = px.scatter_mapbox(final_df, lat='LAT', lon='LON', zoom=5, mapbox_style='carto-positron', hover_name='NAME', hover_data=['Average Assessment Proficiency'], color='Average Assessment Proficiency', color_continuous_midpoint=50)

    # converts the plotly figure to HTML for display
    plotly_html = fig.to_html(full_html=False)

    # renders the HTML template using the variables and dfs created
    return render_template('result.html', schools_data=final_df, plotly_html=plotly_html, state_name=state_name, state_crt=state_crt)

@app.route('/district/<district_name>')
def district_page(district_name): # renders a page when someone clicks on a district on the second page
    state_name = session.get("state_name") # retrieves the state name from the session
    selected_state = session.get("selected_state") # retrieves the codon from the session
    state_crt = session.get("state_crt") # retrieves the CRT status
    schools_data = myProjectDataAnalysisFunctions.query_schools_database(selected_state) # generates the same df as in the previous page
    prof = pd.read_csv(f"{state_name}_df.csv") # generates the same df as the previous page
    final_df = pd.merge(schools_data, prof, on='NAME', how='left') # merges them as in the previous page
    district_df = final_df.loc[final_df["NAME"] == district_name]  # extracts just the row from the district chosen
    
    # creates a plotly figure
    fig = px.scatter_mapbox(district_df, lat='LAT', lon='LON', zoom=5, mapbox_style='carto-positron', hover_name='NAME', hover_data=["Average Assessment Proficiency"], color='Average Assessment Proficiency', color_continuous_midpoint=50)

    # convert the plotly figure to HTML for display
    plotly_html = fig.to_html(full_html=False)
    
    # renders the HTML template using the variables and dfs created
    return render_template('district_page.html', schools_data=district_df, plotly_html=plotly_html, state_name=state_name, state_crt=state_crt)

if __name__ == '__main__':
    app.run(port=5000)


#### There's a lot going on, so we'll break it down step-by-step:

In [6]:
from myProject import *
from flask import Flask, render_template, request, session
import pandas as pd
import sqlite3
import plotly.express as px
import plotly.io as pio
pio.renderers.default="iframe"


app = Flask(__name__) #creates Flask app
app.secret_key = "PIC16B" #sets secret key

myProjectDataAnalysisFunctions.generate_state_districts_db() # generates the database

#### This first block of code imports a few things: first, it imports the two other .py files and their respective functions. Then, it imports flask, pandas, sqlite3 (SQL), and plotly, which all are going to be used for their pretty standard functions.

#### Then, we initialize our Flask app, give it a session key (which will be important later), and initialize our database using the function described earlier.

In [7]:
@app.route('/')
def index(): # renders the initial page
    return render_template('index.html', PageTitle="School Data")

@app.route('/process', methods=['POST'])
def process(): # renders the page based on what state is chosen on the first page
    selected_state = request.form['state'] # retrieves the chosen state
    state_name = selected_state # sets a variable equal to that state
    session["state_name"] = state_name # adds it to the session
    selected_state = myProjectFlaskFunctions.state_code_conversion(selected_state) # converts the state name to its 2 letter codon
    session["selected_state"] = selected_state # adds the codon to the session
    state_crt = myProjectFlaskFunctions.state_crt_condition(state_name) # retrieves the CRT status for the state
    session["state_crt"] = state_crt # adds the CRT status to the session
    schools_data = myProjectDataAnalysisFunctions.generate_state_districts_df(selected_state) # creates a dataframe for the given state from the database
    prof = pd.read_csv(f"{state_name}_df.csv") # reads in the already-formatted state database for the given state
    final_df = pd.merge(schools_data, prof, on='NAME', how='left') # merges the dfs

    # creates a plotly figure
    fig = px.scatter_mapbox(final_df, lat='LAT', lon='LON', zoom=5, mapbox_style='carto-positron', hover_name='NAME', hover_data=['Average Assessment Proficiency'], color='Average Assessment Proficiency', color_continuous_midpoint=50)

    # converts the plotly figure to HTML for display
    plotly_html = fig.to_html(full_html=False)

    # renders the HTML template using the variables and dfs created
    return render_template('result.html', schools_data=final_df, plotly_html=plotly_html, state_name=state_name, state_crt=state_crt)

#### This section initializes two things: our index page (which is the first page on the Flask site, and the first one you see), and codes the process for our second page, the process page.

#### Let's talk about the index page. It looks very simple, but it is written from an HTML file that is actually a little complicated. It looks like this:

<!DOCTYPE html>
<html>
<head>
    <title>Select State</title>
</head>
<body>
    <form action="/process" method="post">
        <label for="state">Select State:</label>
        <select id="state" name="state">
            <option value="Alabama">Alabama</option>
            <option value="Alaska">Alaska</option>
            <option value="Arizona"> Arizona</option>
            <option value="Arkansas">Arkansas</option>
            <option value="California">California</option>
            <option value="Colorado">Colorado</option>
            <option value="Connecticut">Connecticut</option>
            <option value="Delaware">Delaware</option>
            <option value="Florida">Florida</option>
            <option value="Georgia">Georgia</option>
            <option value="Hawaii">Hawaii</option>
            <option value="Idaho">Idaho</option>
            <option value="Illinois">Illinois</option>
            <option value="Indiana">Indiana</option>
            <option value="Iowa">Iowa</option>
            <option value="Kansas">Kansas</option>
            <option value="Kentucky">Kentucky</option>
            <option value="Louisiana">Louisiana</option>
            <option value="Maine">Maine</option>
            <option value="Maryland">Maryland</option>
            <option value="Massachusetts">Massachusetts</option>
            <option value="Michigan">Michigan</option>
            <option value="Minnesota">Minnesota</option>
            <option value="Mississippi">Mississippi</option>
            <option value="Missouri">Missouri</option>
            <option value="Montana">Montana</option>
            <option value="Nebraska">Nebraska</option>
            <option value="Nevada">Nevada</option>
            <option value="New Hampshire">New Hampshire</option>
            <option value="New Jersey">New Jersey</option>
            <option value="New Mexico">New Mexico</option>
            <option value="New York">New York</option>
            <option value="North Carolina">North Carolina</option>
            <option value="North Dakota">North Dakota</option>
            <option value="Ohio">Ohio</option>
            <option value="Oklahoma">Oklahoma</option>
            <option value="Oregon">Oregon</option>
            <option value="Pennsylvania">Pennsylvania</option>
            <option value="Rhode Island">Rhode Island</option>
            <option value="South Carolina">South Carolina</option>
            <option value="South Dakota">South Dakota</option>
            <option value="Tennessee">Tennessee</option>
            <option value="Texas">Texas</option>
            <option value="Utah">Utah</option>
            <option value="Vermont">Vermont</option>
            <option value="Virginia">Virginia</option>
            <option value="Washington">Washington</option>
            <option value="West Virginia">West Virginia</option>
            <option value="Wisconsin">Wisconsin</option>
            <option value="Wyoming">Wyoming</option>
        </select>
        <input type="submit" value="Submit">
    </form>
</body>
</html>


#### It contains a prompt: select state. Then, it contains the information to encode a dropdown menu which includes all 50 states. Once you click a state, it is submitted to the site. This submission causes a secondary page to be created.

#### Now, let's talk about the process (or "result") page. It looks like this:

<!DOCTYPE html>
<html>
<head>
    <title>District Data for {{ state_name }}</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        body {
            text-align: center;
        }

        h1 {
            margin-top: 50px;
        }
    </style>
</head>
<body>
    <h1>District Information for {{ state_name }} </h1>
    <h2>CRT status: {{ state_crt }}
    <div id="map">{{ plotly_html|safe }}</div>
    <table border="1">
        <thead>
            <tr>
                <th>District</th>
                <th>Proficiency</th>
            </tr>
        </thead>
        <tbody>
            {% for index, row in schools_data.iterrows() %}
            <tr>
                <td><a href="{{ url_for('district_page', district_name=row['NAME']) }}">{{ row['NAME'] }}</a></td>
                <td>{{ row['Average Assessment Proficiency'] }}</td>
            </tr>
            {% endfor %}
        </tbody>
    </table>
    
    <p>For entries that say "nan", there is no data on their proficiency. Proficiency is measured based on the percent of students that reach a desired score on state testing. </p>

    <div id="map">{{ plotly_html|safe }}</div>
</body>
</html>

#### This one is even more complicated than the other page. First, it has a title that has some formatting information to make it look nicer. Then, it has a header, which incorporates the variable `state_name` through some HTML-specific syntax. It also has a link to allow for the embedding of the plotly graph, which comes later. It has a subheading which displays whether or not the state has banned CRT or not.

#### Then, it has a div for the plotly map, which is the cornerstone of the webpage. After that, it has a table, which has the school districts and their proficiency scores all labeled. The most interesting part of the page is in the table: each row has a reference for the url for each district's individual webpage. So, each district name in the table becomes a hyperlink because of the `<a href>` tag. Then there's a column for their scores.

#### Then, there's a paragraph with a disclaimer in a `<p>` tag, and then finally the actual plotly graph, which displays at the top. All of these combine to make a webpage with an interactable plotly graph, a data table with hyperlinks, and a disclaimer paragraph at the bottom.

#### The bits of the code that contain the definitions for all the data is below:

In [8]:
@app.route('/process', methods=['POST'])
def process(): # renders the page based on what state is chosen on the first page
    selected_state = request.form['state'] # retrieves the chosen state
    state_name = selected_state # sets a variable equal to that state
    session["state_name"] = state_name # adds it to the session
    selected_state = myProjectFlaskFunctions.state_code_conversion(selected_state) # converts the state name to its 2 letter codon
    session["selected_state"] = selected_state # adds the codon to the session
    state_crt = myProjectFlaskFunctions.state_crt_condition(state_name) # retrieves the CRT status for the state
    session["state_crt"] = state_crt # adds the CRT status to the session
    schools_data = myProjectDataAnalysisFunctions.generate_state_districts_df(selected_state) # creates a dataframe for the given state from the database
    prof = pd.read_csv(f"{state_name}_df.csv") # reads in the already-formatted state database for the given state
    final_df = pd.merge(schools_data, prof, on='NAME', how='left') # merges the dfs

    # creates a plotly figure
    fig = px.scatter_mapbox(final_df, lat='LAT', lon='LON', zoom=5, mapbox_style='carto-positron', hover_name='NAME', hover_data=['Average Assessment Proficiency'], color='Average Assessment Proficiency', color_continuous_midpoint=50)

    # converts the plotly figure to HTML for display
    plotly_html = fig.to_html(full_html=False)

    # renders the HTML template using the variables and dfs created
    return render_template('result.html', schools_data=final_df, plotly_html=plotly_html, state_name=state_name, state_crt=state_crt)

#### There's a lot going on here too. Starting with the `request.form` line, which takes the initial request from the index (that has the state of interest), and then going into a couple lines that define a session variable. This session variable is allowed because of the key from the beginning of the Flask app, and this session variable allows us to transfer variables across pages.

#### Then, we run code to clean up our information (getting the 2 letter codon, getting the CRT condition, and generating a df for the state of interest), and organize it all into one final dataframe.

#### Lastly, we generate a `scatter_mapbox`, which incorporates all the information we just generated, and we convert it to HTML so that it can be displayed on our next page. Then we render our template, passing all the necessary variables to it.

#### Next, we look at our district pages:

In [ ]:
@app.route('/district/<district_name>')
def district_page(district_name): # renders a page when someone clicks on a district on the second page
    state_name = session.get("state_name") # retrieves the state name from the session
    selected_state = session.get("selected_state") # retrieves the codon from the session
    state_crt = session.get("state_crt") # retrieves the CRT status
    schools_data = myProjectDataAnalysisFunctions.query_schools_database(selected_state) # generates the same df as in the previous page
    prof = pd.read_csv(f"{state_name}_df.csv") # generates the same df as the previous page
    final_df = pd.merge(schools_data, prof, on='NAME', how='left') # merges them as in the previous page
    district_df = final_df.loc[final_df["NAME"] == district_name]  # extracts just the row from the district chosen
    
    # creates a plotly figure
    fig = px.scatter_mapbox(district_df, lat='LAT', lon='LON', zoom=5, mapbox_style='carto-positron', hover_name='NAME', hover_data=["Average Assessment Proficiency"], color='Average Assessment Proficiency', color_continuous_midpoint=50)

    # convert the plotly figure to HTML for display
    plotly_html = fig.to_html(full_html=False)
    
    # renders the HTML template using the variables and dfs created
    return render_template('district_page.html', schools_data=district_df, plotly_html=plotly_html, state_name=state_name, state_crt=state_crt)

#### The main highlights of this block of code is that the variables from the process route are being carried over by the `session.get` lines of code. Then, the rest is pretty recognizable, except we convert the final_df to a district_df which only contains the district of interest. Then, we generate everything else in the same way, and create a very familiar looking page of just our district.

#### We finish our code with one line:

In [ ]:
if __name__ == '__main__':
    app.run(port=5000)

#### This line of code just says that the connection should be on port 5000, and that the Flask site should be initialized when the code is run (on the index page).

#### This project is interesting in that is has a lot of local files. Namely, the files for each of the state data as well as the .csv containing all of the school districts are all local, and are processed locally. The website would be very messy, resource-intensive, and slow if all the data was hosted and processed through the site, so a lot of the work is done on the local side in order to ensure the website is as smooth and quick to use as possible.

#### Although the information for most of the states is not present, the initial project framework represents an overwhelming majority of the problem-solving required, and the state data acquisition is a slow, lengthy process that gets very repetitive. So, while this project is ultimately unfinished (from its original goal), a very solid framework has been constructed to put everything together!